In [1]:
import os
import glob
import math
from pathlib import Path
import datetime
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from mpl_toolkits.axes_grid1 import make_axes_locatable

import dotenv

In [2]:
# pd.set_option('display.max_columns', 20)
pd.set_option('display.max_columns', None)

In [3]:
# Navigate one level up to the main repository (or where you have your .env file)
os.chdir('..')

In [4]:
# Read environmental variables
env_file = os.path.join('.env') 
dotenv.load_dotenv(env_file)

data_dir = os.getenv('DATA_PATH')
input_dir = os.getenv('INPUT_PATH')
out_dir = os.getenv('OUTPUT_PATH')

In [6]:
model_run_name = "slf_ensemble_calibrated"
countries_file_name = "countries_ensembleHost"

results_dir = out_dir + model_run_name
native_countries_list = ["China", "Viet Nam"]

In [10]:
param_vals = [os.path.basename(name) for name in glob.glob(results_dir + '/*')]
param_vals

['year2005_alpha0.4127_lamda1.9859_6802-6802',
 'year2005_alpha0.4271_lamda1.7617_6802-6802',
 'year2005_alpha0.4472_lamda2.1258_6802-6802',
 'year2005_alpha0.4875_lamda0.7315_6802-6802',
 'year2005_alpha0.4935_lamda1.0523_6802-6802',
 'year2005_alpha0.4991_lamda2.7723_6802-6802',
 'year2005_alpha0.5086_lamda2.7224_6802-6802',
 'year2005_alpha0.5112_lamda2.143_6802-6802',
 'year2005_alpha0.5184_lamda1.683_6802-6802',
 'year2005_alpha0.5245_lamda1.5476_6802-6802',
 'year2005_alpha0.524_lamda1.704_6802-6802',
 'year2005_alpha0.5266_lamda2.1221_6802-6802',
 'year2005_alpha0.5319_lamda1.7949_6802-6802',
 'year2005_alpha0.531_lamda1.1323_6802-6802',
 'year2005_alpha0.5381_lamda2.0319_6802-6802',
 'year2005_alpha0.5385_lamda2.4695_6802-6802',
 'year2005_alpha0.5441_lamda2.1417_6802-6802',
 'year2005_alpha0.5454_lamda1.8082_6802-6802',
 'year2005_alpha0.5472_lamda2.2797_6802-6802',
 'year2005_alpha0.5479_lamda2.3479_6802-6802',
 'year2005_alpha0.5495_lamda1.4142_6802-6802',
 'year2005_alpha0.

In [14]:
sim_names = [os.path.basename(name) for name in glob.glob(results_dir + '/*/run*')]
len(sim_names)

1000

In [ ]:
countries_path = f"{input_dir}/{countries_file_name}.gpkg"
countries_geo = geopandas.read_file(countries_path)

In [17]:
for i in range(0, len(sim_names)):
    print(i)
    param_val = param_vals[i]
    model_run_name = sim_names[i]
    countries_path = f"{input_dir}/{countries_file_name}.gpkg"
    countries_geo = geopandas.read_file(countries_path)
    countries = countries_geo.iloc[:,[4]]
    countries.set_index("NAME")
    countries_firstintro = countries.iloc[:,[0]]
    countries_reintros = countries.iloc[:,[0]]
    org_dest_all = pd.DataFrame()
    
    # paths = Path(results_dir).glob('**/origin_destination.csv')
    paths = Path(results_dir).glob('**/origin_destination.csv')
    for path in paths:
        path_in_str = str(path)
        run = path_in_str.split("\\")[3] ## should be [-2] for parameter sets with multiple runs within the same dir
        org_dest = (pd.read_csv(path)).iloc[:,1:4]
        org_dest["TS"] = org_dest["TS"].astype(str)
        org_dest["TS"] = (org_dest.TS.str[:4]).astype(int)
        org_dest_all = org_dest_all.append(org_dest)
        intros = org_dest.iloc[:,1:4]
        intros = intros.rename(columns={"Destination":"NAME", "TS":run})
        firstintro = intros.drop_duplicates(subset = ["NAME"])
        countries_firstintro = pd.merge(countries_firstintro, firstintro, on="NAME", how="left")
        reintros = intros.groupby("NAME").count()
        countries_reintros = pd.merge(countries_reintros, reintros, on="NAME", how="left")

    countries_firstintro = countries_firstintro.set_index("NAME")
    countries_reintros = countries_reintros.set_index("NAME")
    
    # set country of interest (coi)
    coi = "United States"
    coi_intros = org_dest_all[org_dest_all["Destination"] == coi]
    coi_intros = coi_intros.groupby("Origin").count()[["Destination"]]
    coi_intros = coi_intros.rename(columns={"Destination":"COI source"})

    countries_geo = countries_geo.merge(coi_intros, how="left", left_on="NAME", right_on="Origin")
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0.1)
    ax.set_title("Introduction Sources for " + coi + "\n" + model_run_name, fontsize=18)
    countries_geo.plot(column='COI source', ax=ax, legend=True, legend_kwds={'label': "intro source count"}, missing_kwds={'color': 'lightgrey'}, cax=cax)
    plt.savefig(results_dir + coi + "_intro_sources.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()
    
    arr_yr_mean_all = []
    arr_yr_mode_all = []
    arr_yr_min_all = []
    arr_yr_max_all = []
    arr_yr_range_all = []
    intro_proportion_all = []

    for row in range(len(countries_firstintro.index)):
        runs_no_intro = countries_firstintro.iloc[row].isnull().sum()
        intro_proportion = 1 - (runs_no_intro / len(countries_firstintro.columns))
        intro_proportion_all.append(intro_proportion)

        if intro_proportion == 0:
            arr_yr_min_all.append(None)
            arr_yr_max_all.append(None)
            arr_yr_mean_all.append(None)
            arr_yr_mode_all.append(None)
            arr_yr_range_all.append(None)
            
        else:
            arr_yr_min = countries_firstintro.iloc[row].min()
            arr_yr_min_all.append(arr_yr_min)
            arr_yr_max = countries_firstintro.iloc[row].max()
            arr_yr_max_all.append(arr_yr_max)
            arr_yr_mean = math.floor(np.nanmean(countries_firstintro.iloc[row]))
            arr_yr_mean_all.append(arr_yr_mean)
            arr_yr_mode = countries_firstintro.iloc[row].mode()
            if len(arr_yr_mode) > 1:
                arr_yr_mode = int(arr_yr_mode.mean())
            else:
                arr_yr_mode = arr_yr_mode[0]
            arr_yr_mode_all.append(arr_yr_mode)
            arr_yr_range_all.append(arr_yr_max - arr_yr_min)

    countries_firstintro["arr_yr_mean"] = arr_yr_mean_all
    countries_firstintro["arr_yr_mode"] = arr_yr_mode_all
    countries_firstintro["arr_yr_min"] = arr_yr_min_all
    countries_firstintro["arr_yr_max"] = arr_yr_max_all
    countries_firstintro["arr_yr_range"] = arr_yr_range_all
    countries_firstintro["intro_proportion"] = intro_proportion_all
    countries_firstintro.loc[native_countries_list, 'arr_yr_mean'] = None
    countries_firstintro.loc[native_countries_list, 'arr_yr_mode'] = None
    countries_firstintro.loc[native_countries_list, 'arr_yr_min'] = None
    countries_firstintro.loc[native_countries_list, 'arr_yr_max'] = None
    countries_firstintro.loc[native_countries_list, 'arr_yr_range'] = None
    countries_firstintro.loc[native_countries_list, 'intro_proportion'] = None
    
    countries_reintros = countries_reintros.fillna(0)
    countries_reintros["num_reintros_mean"] = round(countries_reintros.mean(axis=1)).astype(int)
    countries_reintros.loc[native_countries_list, 'num_reintros_mean'] = None
    
    countries_geo = countries_geo.merge(countries_firstintro["arr_yr_mean"], on='NAME')
    countries_geo = countries_geo.merge(countries_firstintro["arr_yr_mode"], on='NAME')
    countries_geo = countries_geo.merge(countries_firstintro["arr_yr_min"], on='NAME')
    countries_geo = countries_geo.merge(countries_firstintro["arr_yr_max"], on='NAME')
    countries_geo = countries_geo.merge(countries_firstintro["arr_yr_range"], on='NAME')
    countries_geo = countries_geo.merge(countries_firstintro["intro_proportion"], on='NAME')
    countries_geo = countries_geo.merge(countries_reintros["num_reintros_mean"], on='NAME')
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0.1)
    ax.set_title("Number of Reintroductions (mean)\n" + model_run_name, fontsize=18)
    countries_geo['num_reintros_mean'] = countries_geo['num_reintros_mean'].replace(0, np.nan)
    countries_geo.plot(column='num_reintros_mean', ax=ax, legend=True, legend_kwds={'label': "reintroductions"}, missing_kwds={'color': 'lightgrey'}, cax=cax)
    plt.savefig(results_dir + "num_reintros.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    plt.title("Year of First Introduction (mean)\n" + model_run_name, fontsize=18)
    countries_geo.plot(column='arr_yr_mean', categorical=True, cmap="viridis", legend=True, ax=ax, missing_kwds={'color': 'lightgrey'}, legend_kwds={'loc': 'lower left'})
    #countries_geo.plot(column='arr_yr_mode', scheme="User_Defined", classification_kwds=dict(bins=[2010,2012,2014,2016,2018,2020]), legend=True, ax=ax, missing_kwds={'color': 'lightgrey'})
    plt.savefig(results_dir + "first_intros_mean.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    plt.title("Year of First Introduction (mode)\n" + model_run_name, fontsize=18)
    countries_geo.plot(column='arr_yr_mode', categorical=True, cmap="viridis", legend=True, ax=ax, missing_kwds={'color': 'lightgrey'}, legend_kwds={'loc': 'lower left'})
    #countries_geo.plot(column='arr_yr_mode', scheme="User_Defined", classification_kwds=dict(bins=[2010,2012,2014,2016,2018,2020]), legend=True, ax=ax, missing_kwds={'color': 'lightgrey'})
    plt.savefig(results_dir + "first_intros_mode.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()
    
    ## First Year (min)
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    plt.title("Year of First Introduction (min)\n" + model_run_name, fontsize=18)
    countries_geo.plot(column='arr_yr_min', categorical=True, cmap="viridis", legend=True, ax=ax, missing_kwds={'color': 'lightgrey'}, legend_kwds={'loc': 'lower left'})
    #countries_geo.plot(column='arr_yr_mode', scheme="User_Defined", classification_kwds=dict(bins=[2010,2012,2014,2016,2018,2020]), legend=True, ax=ax, missing_kwds={'color': 'lightgrey'})
    plt.savefig(results_dir + "first_intros_min.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()
    
    ## First Year (max)    
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    plt.title("Year of First Introduction (max)\n" + model_run_name, fontsize=18)
    countries_geo.plot(column='arr_yr_max', categorical=True, cmap="viridis", legend=True, ax=ax, missing_kwds={'color': 'lightgrey'}, legend_kwds={'loc': 'lower left'})
    #countries_geo.plot(column='arr_yr_mode', scheme="User_Defined", classification_kwds=dict(bins=[2010,2012,2014,2016,2018,2020]), legend=True, ax=ax, missing_kwds={'color': 'lightgrey'})
    plt.savefig(results_dir + "first_intros_max.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()
    
    ## Range
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    plt.title("Year of First Introduction (range)\n" + model_run_name, fontsize=18)
    countries_geo.plot(column='arr_yr_range', categorical=True, cmap="viridis", legend=True, ax=ax, missing_kwds={'color': 'lightgrey'}, legend_kwds={'loc': 'lower left'})
    #countries_geo.plot(column='arr_yr_mode', scheme="User_Defined", classification_kwds=dict(bins=[2010,2012,2014,2016,2018,2020]), legend=True, ax=ax, missing_kwds={'color': 'lightgrey'})
    plt.savefig(results_dir + "first_intros_range.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()
    
    ## Proportion of Runs
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0.1)
    ax.set_title("Proportion of Runs with Introductions\n" + model_run_name, fontsize=18)
    countries_geo['intro_proportion'] = countries_geo['intro_proportion'].replace(0, np.nan)
    countries_geo.plot(column='intro_proportion', ax=ax, legend=True, legend_kwds={'label': "proportion"}, missing_kwds={'color': 'lightgrey'}, cax=cax)
    plt.savefig(results_dir + "intro_proportion.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()    
    

0


<ipython-input-17-991ff2cc7e17>:25: FutureWarning: Passing 'suffixes' which cause duplicate columns {'slf_model_x'} in the result is deprecated and will raise a MergeError in a future version.
  countries_firstintro = pd.merge(countries_firstintro, firstintro, on="NAME", how="left")
<ipython-input-17-991ff2cc7e17>:27: FutureWarning: Passing 'suffixes' which cause duplicate columns {'slf_model_x'} in the result is deprecated and will raise a MergeError in a future version.
  countries_reintros = pd.merge(countries_reintros, reintros, on="NAME", how="left")


1


<ipython-input-17-991ff2cc7e17>:25: FutureWarning: Passing 'suffixes' which cause duplicate columns {'slf_model_x'} in the result is deprecated and will raise a MergeError in a future version.
  countries_firstintro = pd.merge(countries_firstintro, firstintro, on="NAME", how="left")
<ipython-input-17-991ff2cc7e17>:27: FutureWarning: Passing 'suffixes' which cause duplicate columns {'slf_model_x'} in the result is deprecated and will raise a MergeError in a future version.
  countries_reintros = pd.merge(countries_reintros, reintros, on="NAME", how="left")


KeyboardInterrupt: 

In [ ]:
for i in range(0, len(sim_names)):
    print(i)
    model_run_name = sim_names[i]
    print(f"\t{model_run_name}")
    results_dir = f"slf_model/outputs/{model_run_name}/"
    native_countries_list = ["China", "Viet Nam"]
    
    countries_path = "Q:/Shared drives/Pandemic Data/slf_model/inputs/countries_ensembleHost.gpkg"
    countries_geo = geopandas.read_file(countries_path)
    countries = countries_geo.iloc[:,[4]]
    countries.set_index("NAME")
    countries_firstintro = countries.iloc[:,[0]]
    countries_reintros = countries.iloc[:,[0]]
    org_dest_all = pd.DataFrame()
    
    paths = Path(results_dir).glob('**/origin_destination.csv')
    for path in paths:
        path_in_str = str(path)
        run = path_in_str.split("\\")[-2]
        org_dest = (pd.read_csv(path)).iloc[:,1:4]
        org_dest["TS"] = org_dest["TS"].astype(str)
        org_dest["TS"] = (org_dest.TS.str[:4]).astype(int)
        org_dest_all = org_dest_all.append(org_dest)
        intros = org_dest.iloc[:,1:4]
        intros = intros.rename(columns={"Destination":"NAME", "TS":run})
        firstintro = intros.drop_duplicates(subset = ["NAME"])
        countries_firstintro = pd.merge(countries_firstintro, firstintro, on="NAME", how="left")
        reintros = intros.groupby("NAME").count()
        countries_reintros = pd.merge(countries_reintros, reintros, on="NAME", how="left")

    countries_firstintro = countries_firstintro.set_index("NAME")
    countries_reintros = countries_reintros.set_index("NAME")
    
    # set country of interest (coi)
    coi = "United States"
    coi_intros = org_dest_all[org_dest_all["Destination"] == coi]
    coi_intros = coi_intros.groupby("Origin").count()[["Destination"]]
    coi_intros = coi_intros.rename(columns={"Destination":"COI source"})

    countries_geo = countries_geo.merge(coi_intros, how="left", left_on="NAME", right_on="Origin")
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0.1)
    ax.set_title("Introduction Sources for " + coi + "\n" + model_run_name, fontsize=18)
    countries_geo.plot(column='COI source', ax=ax, legend=True, legend_kwds={'label': "intro source count"}, missing_kwds={'color': 'lightgrey'}, cax=cax)
    plt.savefig(results_dir + coi + "_intro_sources.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()
    
    arr_yr_mean_all = []
    arr_yr_mode_all = []
    arr_yr_min_all = []
    arr_yr_max_all = []
    arr_yr_range_all = []
    intro_proportion_all = []

    for row in range(len(countries_firstintro.index)):
        runs_no_intro = countries_firstintro.iloc[row].isnull().sum()
        intro_proportion = 1 - (runs_no_intro / len(countries_firstintro.columns))
        intro_proportion_all.append(intro_proportion)

        if intro_proportion == 0:
            arr_yr_min_all.append(None)
            arr_yr_max_all.append(None)
            arr_yr_mean_all.append(None)
            arr_yr_mode_all.append(None)
            arr_yr_range_all.append(None)
            
        else:
            arr_yr_min = countries_firstintro.iloc[row].min()
            arr_yr_min_all.append(arr_yr_min)
            arr_yr_max = countries_firstintro.iloc[row].max()
            arr_yr_max_all.append(arr_yr_max)
            arr_yr_mean = math.floor(np.nanmean(countries_firstintro.iloc[row]))
            arr_yr_mean_all.append(arr_yr_mean)
            arr_yr_mode = countries_firstintro.iloc[row].mode()
            if len(arr_yr_mode) > 1:
                arr_yr_mode = int(arr_yr_mode.mean())
            else:
                arr_yr_mode = arr_yr_mode[0]
            arr_yr_mode_all.append(arr_yr_mode)
            arr_yr_range_all.append(arr_yr_max - arr_yr_min)

    countries_firstintro["arr_yr_mean"] = arr_yr_mean_all
    countries_firstintro["arr_yr_mode"] = arr_yr_mode_all
    countries_firstintro["arr_yr_min"] = arr_yr_min_all
    countries_firstintro["arr_yr_max"] = arr_yr_max_all
    countries_firstintro["arr_yr_range"] = arr_yr_range_all
    countries_firstintro["intro_proportion"] = intro_proportion_all
    countries_firstintro.at[native_countries_list, 'arr_yr_mean'] = None
    countries_firstintro.at[native_countries_list, 'arr_yr_mode'] = None
    countries_firstintro.at[native_countries_list, 'arr_yr_min'] = None
    countries_firstintro.at[native_countries_list, 'arr_yr_max'] = None
    countries_firstintro.at[native_countries_list, 'arr_yr_range'] = None
    countries_firstintro.at[native_countries_list, 'intro_proportion'] = None
    
    countries_reintros = countries_reintros.fillna(0)
    countries_reintros["num_reintros_mean"] = round(countries_reintros.mean(axis=1)).astype(int)
    countries_reintros.at[native_countries_list, 'num_reintros_mean'] = None
    
    countries_geo = countries_geo.merge(countries_firstintro["arr_yr_mean"], on='NAME')
    countries_geo = countries_geo.merge(countries_firstintro["arr_yr_mode"], on='NAME')
    countries_geo = countries_geo.merge(countries_firstintro["arr_yr_min"], on='NAME')
    countries_geo = countries_geo.merge(countries_firstintro["arr_yr_max"], on='NAME')
    countries_geo = countries_geo.merge(countries_firstintro["arr_yr_range"], on='NAME')
    countries_geo = countries_geo.merge(countries_firstintro["intro_proportion"], on='NAME')
    countries_geo = countries_geo.merge(countries_reintros["num_reintros_mean"], on='NAME')
      
    fig, ax = plt.subplots(1, 1, figsize=(20,20))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0.1)
    ax.set_title("Year of First Introduction (runs with intros >50%)\n" + model_run_name, fontsize=18)
    countries_geo['arr_yr_min'] = np.where(countries_geo['intro_proportion'] < 0.5, np.nan, countries_geo['arr_yr_min'])
    countries_geo.plot(column='arr_yr_min', ax=ax, legend=True, legend_kwds={'label': "proportion"}, missing_kwds={'color': 'lightgrey'}, cax=cax)
    plt.savefig(results_dir + "first_intro_min_more50pct.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0.1)
    ax.set_title("Year of First Introduction (mean of runs with intros >50%)\n" + model_run_name, fontsize=18)
    countries_geo['arr_yr_mean'] = np.where(countries_geo['intro_proportion'] < 0.5, np.nan, countries_geo['arr_yr_mean'])
    countries_geo.plot(column='arr_yr_mean', ax=ax, legend=True, legend_kwds={'label': "proportion"}, missing_kwds={'color': 'lightgrey'}, cax=cax)
    plt.savefig(results_dir + "first_intro_mean_more50pct.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 20))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2%", pad=0.1)
    ax.set_title("Runs with Introductions (>50%)\n" + model_run_name, fontsize=18)
    countries_geo['intro_proportion'] = np.where(countries_geo['intro_proportion'] < 0.5, np.nan, countries_geo['intro_proportion'])
    countries_geo.plot(column='intro_proportion', ax=ax, legend=True, legend_kwds={'label': "proportion"}, missing_kwds={'color': 'lightgrey'}, cax=cax)
    plt.savefig(results_dir + "intro_proportion_more50pct.png", bbox_inches='tight', pad_inches = 0.01)
    plt.close()

0
	year2005_alpha0.7_beta0.5_lamda1.7_6802
1
	year2005_alpha0.75_beta0.5_lamda1.7_6802
2
	year2005_alpha0.6_beta0.5_lamda2.4_6802
